Laplace mixed boundary values
==============================

WIP

**keys**: homogeneous Laplace bvp, Calderon projector, Dirichlet data, Neumann data 

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.krylovspace import CG, GMRes
from ngsolve.webgui import Draw
from ngsolve.bem import *

We consider an interior boundary value problem with mixed boundary conditions like this:

|  |  |  |
| -|--|- |
| $ \begin{array}{r rcl r} & \Delta u &=& 0  &\mathrm{in}\; \Omega\,,\\ \textnormal{Dirichlet condition} & \gamma_0 u &=& u_0 & \mathrm{on}\; \Gamma_0\,,\\ \textnormal{Neumann condition} & \gamma_1 u &=&  u_1 & \mathrm{on}\; \Gamma_1\,. \end{array} $   | $\quad\quad\quad$  | ![](resources/BEM_Mixed.png)  |

In [ ]:
topsphere = Sphere((0,0,0), 1) * Box((-1,-1,0),(1,1,1))
botsphere = Sphere((0,0,0), 1) - Box((-1,-1,0),(1,1,1))
topsphere.faces.name = "neumann"
botsphere.faces.name = "dirichlet"
shape = Fuse( [topsphere,botsphere] )

order = 3
mesh = Mesh(OCCGeometry(shape).GenerateMesh(maxh=0.25)).Curve(order)
Draw (mesh);

The following representation formula for the solution $u$ holds:

$$ x \in \Omega: \quad u(x) =  \displaystyle{ \int\limits_\Gamma} \displaystyle{\frac{1}{4\,\pi}\, \frac{1}{\| x-y\|} } \, \gamma_1 u (y)\, \mathrm{d}\sigma_y - \displaystyle{ \int\limits_\Gamma} \displaystyle{\frac{1}{4\,\pi}\, \frac{\langle n(y) , x-y\rangle }{\| x-y\|^3} } \, \gamma_0 u (y)\, \mathrm{d}\sigma_y\,. $$ 

The Calderon projector relates the Dirichlet and the Neumann traces of the solution $u$, i.e., 

  
$$  \left( \begin{array}{c} \gamma_0 u \\ \gamma_1 u \end{array}\right) =  \left( \begin{array}{cc} V & \frac12 - K \\ \frac12 + K^\intercal & D \end{array} \right)  \left( \begin{array}{c} \gamma_1 u \\ \gamma_0 u \end{array}\right)\,, $$

and we use it to solve for the Dirchlet trace on $\Gamma_1$ and the Neumann trace on $\Gamma_0$. 

Define the finite element spaces for the Dirichlet and the Neumann data:

In [ ]:
# use L2 conform elements for the Neumann trace, label all dofs where Neumann data is given:
fesL2 = SurfaceL2(mesh, order=order-1, dirichlet="neumann")
u,v = fesL2.TnT()

# use H1 conform elements for the Dirichlet trace, label all dofs where Dirichlet data is given:
fesH1 = H1(mesh, order=order, dirichlet="dirichlet", definedon=mesh.Boundaries(".*"))
u1,v1 = fesH1.TnT()

print ("ndofL2 = ", fesL2.ndof, "ndofH1 = ", fesH1.ndof)

Compute and set the Dirichlet and the Neumann data:

In [ ]:
uexa = CF(x)
ud = GridFunction(fesH1)
ud.Interpolate(uexa, definedon=mesh.Boundaries("dirichlet"))

n = specialcf.normal(3)
gradn_uexa = CF((uexa.Diff(x), uexa.Diff(y), uexa.Diff(z))) * n
un = GridFunction(fesL2)
un.Interpolate(gradn_uexa, definedon=mesh.Boundaries("neumann"))

Draw(ud, mesh, draw_vol=False);

Compute boundary integral operators $\mathrm{V}, \mathrm{K}, \mathrm{W}$ and the mass matrix $\mathrm{M}$:

In [ ]:
intorder = 2 * order + 6
with TaskManager():
    V = SingleLayerPotentialOperator(fesL2, intorder=intorder)
    K = DoubleLayerPotentialOperator(fesH1, fesL2, intorder=intorder)
    W = HypersingularOperator(fesH1, intorder=intorder)
    M = BilinearForm(u1.Trace() * v.Trace() * ds(bonus_intorder=3)).Assemble()

Insert all given data in the Dirichlet-to-Neumann map and compute the right hand side vector: 

In [ ]:
with TaskManager():
    fd = ((0.5 * M.mat + K.mat) * ud.vec - V.mat * un.vec).Evaluate()
    fn = ((0.5 * M.mat.T - K.mat.T) * un.vec - W.mat * ud.vec).Evaluate()

Solve for the missing trace data:

In [ ]:
with TaskManager():
    pred = BilinearForm(u.Trace() * v.Trace() * ds(bonus_intorder=3), check_unused=False).Assemble()
    pren = BilinearForm(u1.Trace() * v1.Trace() * ds(bonus_intorder=3), check_unused=False).Assemble()

lhs = BlockMatrix([[V.mat, - K.mat], [K.mat.T, W.mat]])
rhs = BlockVector([fd, fn])
pre = BlockMatrix ([[pred.mat.Inverse(freedofs=fesL2.FreeDofs()), None], [None, pren.mat.Inverse(freedofs=fesH1.FreeDofs())]])

sol = GMRes(A=lhs, b=rhs, pre=pre, tol=1e-8, maxsteps=50)

Have a look at the Neumann data on $\Gamma_0$ and compute the error:

In [ ]:
gfu = GridFunction(fesL2)
gfu.vec[:] = sol[0]
print ("L2 error of surface gradients =", sqrt(Integrate((gradn_uexa - gfu -un)**2, mesh.Boundaries(".*"), BND)))
Draw(gfu)

Have a look at the Dirichlet data on $\Gamma_1$ and compute the error:

In [ ]:
gfu = GridFunction(fesH1)
gfu.vec[:] = sol[1]
print ("L2 error Dirichlet =", sqrt(Integrate((uexa - gfu -ud)**2, mesh.Boundaries(".*"), BND)))
Draw(gfu)